# 🧮 Evolutionary game theory of diatom size in fluctuating environments
This notebook solves a coupled ODE model of competing phytoplankton species of
varying sizes, using the allometric quota scheme of Litchmann *et al.* (2008) with some modifications to the forcing regime.

To use the model, start by selecting ***Run All Cells*** under the ***Run*** menu at the top left of this page.

The rationale and key parameters are explained in [this discussion of Litchman *et al.*'s model](Litchman_etal2009.md).

In [ ]:
# Load modules and set graphics environment
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
#plt.ion();
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from math import *
from Litchman2009 import diatoms

In [ ]:
# Instantiate a Params object and a GUI to modify parameters. 
#params=Params(auto_plot=True)
h = '27px'
n_sizes=widgets.IntText(value=8,width=10,description = r"$n_{sizes}$")
L_n_sizes = widgets.Label(value='Number size classes',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="100%",height=h))
min_size=widgets.FloatText(value=2.5,
                               description = r"$S_{min} (log_{10})$")
L_min_size = widgets.Label(value='Min. cell size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="100%",height=h))
max_size=widgets.FloatText(value=2.5+1.25*8,
                               description = r"$S_{max} (log_{10})$")
L_max_size = widgets.Label(value='Max cell size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="100%",height=h))
qinit=widgets.FloatText(value=0.1,description = r"$q_{init}$")
L_qinit = widgets.Label(value='Initial fractional quota',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
m=widgets.FloatText(value=0.08,description = r"$m$")
L_m = widgets.Label(value='Mortality rate ($day^{-1}$)',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
z_m=widgets.FloatText(value=25,description = r"$z_m$")
L_z_m = widgets.Label(value='Mixed layer depth ($m$)',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
a=widgets.FloatText(value=0.3,description = r"$a$")
L_a = widgets.Label(value='Fractional mixing',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
Rdeep=widgets.FloatText(value=40.,description = r"$R_{deep}$")
L_Rdeep = widgets.Label(value='Deep nitrogen',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
ui1 = widgets.VBox([n_sizes,min_size,max_size,m])
ui2 = widgets.VBox([a,Rdeep,z_m])
uiL1 = widgets.VBox([L_n_sizes,L_min_size,L_max_size,L_m])
uiL2 = widgets.VBox([L_a,L_Rdeep,L_z_m])

In [ ]:
isizes=widgets.Text(value='all',description = r"$i$")
L_isizes = widgets.Label(value='Size classes ($S_i$) to seed',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
Nseed=widgets.FloatText(value=10,description = r"$N_i$")
L_Nseed = widgets.Label(value='Number cells to seed',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
t_mix=widgets.FloatText(value=8.,description = r"$t_{mix}$")
L_t_mix = widgets.Label(value='Mixing interval ($days$)',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
n_pers=widgets.IntText(value=16,description = r"$n_{pers}$")
L_n_pers = widgets.Label(value='Number of intervals',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
n_record=widgets.IntText(value=12,description = r"$f_2$")
L_n_record = widgets.Label(value='Data per interval',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="100%",height=h))
ui3 = widgets.VBox([qinit,isizes,Nseed])
ui4 = widgets.VBox([t_mix,n_pers,n_record])
uiL3 = widgets.VBox([L_qinit,L_isizes,L_Nseed])
uiL4 = widgets.VBox([L_t_mix,L_n_pers,L_n_record])

uis12 = widgets.HBox([uiL1,ui1,ui2,uiL2])
uis3 = widgets.HBox([uiL3,ui3])
uis4 = widgets.HBox([ui4,uiL4])

global n_sizes_,min_size_,max_size_,m_,a_,Rdeep_,z_m_
def setup_pars(n_sizes,min_size,max_size,m,a,Rdeep,z_m):
    global n_sizes_,min_size_,max_size_,m_,a_,Rdeep_,z_m_
    # Put current parameters into global equivalents for later use
    n_sizes_ = n_sizes
    min_size_ = min_size
    max_size_ = max_size
    m_ = m
    a_ = a
    Rdeep_ = Rdeep
    z_m_ = z_m

out12 = widgets.interactive_output(setup_pars,{'n_sizes':n_sizes,'min_size':min_size,'max_size':max_size,
                                          'm':m,'a':a,'Rdeep':Rdeep,'z_m':z_m})
display(uis12,out12)

:::{figure} #dess_1
:placeholder: ./Images/DESS_1.png
:align: left
:::

In [ ]:
# Set up a new diatoms object, using current parameters
global D
buttonD = widgets.Button(
    value=False,
    description='Set up population',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to set up a simulation model of a diatom population, using the current parameters.',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
outputD = widgets.Output()
@outputD.capture()
def setup(b):
    global D, n_sizes_,min_size_,max_size_,m_,a_,Rdeep_,z_m_
    # Initialize diatoms object
    D = diatoms(z_m=z_m_,n_sizes=n_sizes_,size_range=[min_size_,max_size_],
                m=m_,a=a_,Rdeep=Rdeep_,static_v=False)
    D.setup()
    print('New diatom population initialized...')
buttonD.on_click(setup)
widgets.VBox([buttonD,outputD])

:::{figure} #dess_2
:placeholder: ./Images/DESS_2.png
:align: left
:::

In [ ]:
global qinit_,isizes_,Nseed_
def seed_pars(qinit,isizes,Nseed):
    global qinit_,isizes_,Nseed_
    # Place parameters for seeding cells into global variables for later use.
    # This is executed on each parameter change.
    qinit_ = qinit
    isizes_ = isizes
    Nseed_ = Nseed

out3 = widgets.interactive_output(seed_pars,{'qinit':qinit,'isizes':isizes,'Nseed':Nseed})
display(uis3,out3)

:::{figure} #dess_3
:placeholder: ./Images/DESS_3.png
:align: left
:::

In [ ]:
buttonS = widgets.Button(
    value=False,
    description='Seed cell population',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to add cells to the current population using the current parameters.',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
outputS = widgets.Output()
@outputS.capture()
def seed(b):
    global D,qint_,isizes_,Nseed_
    # Seed selected size class(es), using the parameters set in seed_pars.
    # This is executed only when the button is clicked. 
    # isizes=='all' means all size classes
    if isizes_ == 'all':
        D.seed(qinit=qinit_,N=Nseed_,sizes=[])
        print(f'Seeding all size classes with {Nseed_} cells per liter')
    else:
        D.seed(qinit=qinit_,sizes=list(eval(isizes_)),N=Nseed_)
        print(f'Seeding selected size classes: {isizes_} with {Nseed_} cells per liter')
buttonS.on_click(seed)
widgets.VBox([buttonS,outputS])

:::{figure} #dess_4
:placeholder: ./Images/DESS_4.png
:align: left
:::

In [ ]:
global t_mix_,n_pers_,n_record_
def solve_pars(t_mix,n_pers,n_record):
    global t_mix_,n_pers_,n_record_
    # Record ODE solving parameters for later use; this is executed each time
    # a parameter is changed
    t_mix_ = t_mix
    n_pers_ = n_pers
    n_record_ = n_record
    
out4 = widgets.interactive_output(solve_pars,{'t_mix':t_mix,'n_pers':n_pers,'n_record':n_record})
display(uis4,out4)

:::{figure} #dess_5
:placeholder: ./Images/DESS_5.png
:align: left
:::

In [ ]:
# Set up button to run simulation with current parameters
buttonR = widgets.Button(
    value=False,
    description='Run simulation',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to run the simulation from the current state, using the current parameters.',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
outputR = widgets.Output()
@outputR.capture()
def solve(b):
    global D,t_mix_,n_pers_,n_record_
    # clear previous output
    outputR.clear_output()
    # Seed selected size class(es); isizes==[] means all size classes
    print('Running simulation...')
    D.solveODEs(t_mix=t_mix_,n_pers=n_pers_,n_record=n_record_)
    print('...done.')
    # Display output figures
    D.fig1.show()
    D.fig2.show()
    # Save images to files in png and svg formats
    D.fig1.savefig('period.png')
    D.fig1.savefig('period.svg')
    D.fig2.savefig('averaged.png')
    D.fig2.savefig('averaged.svg')
    #Image('period.png')

buttonR.on_click(solve)
widgets.VBox([buttonR,outputR])


:::{figure} #dess_6
:placeholder: ./Images/DESS_6.png
:align: left
:::

:::{figure} #r1
:placeholder: ./period.png
:align: left
:::

:::{figure} #r2
:placeholder: ./averaged.png
:align: left
:::